# IGN HD liDAR PCD Generation

In [1]:
import numpy as np
import pandas as pd
import pdal
import json
import open3d as o3d
from pyproj import Transformer
import matplotlib.pyplot as plt

In [2]:
input_file = "dem-data/LHD_FXX_0982_6524_PTS_C_LAMB93_IGN69.copc.laz"
# Define a PDAL pipeline to read the file
pipeline = {
    "pipeline": [
        {
            "type": "readers.las",
            "filename": input_file
        }
    ]
}
# Convert the pipeline to JSON format
pipeline_json = json.dumps(pipeline)
# Initialize the PDAL pipeline
p = pdal.Pipeline(pipeline_json)
# Execute the pipeline
count = p.execute()
# Extract the point cloud data
pcd = p.arrays
df_ign = pd.DataFrame(pcd[0])
df_ign.head()

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,Synthetic,KeyPoint,Withheld,Overlap,ScanAngleRank,UserData,PointSourceId,GpsTime,ScanChannel
0,982086.51,6523780.07,1617.46,401,3,3,1,0,2,0,0,0,0,21.204000,7,5264,3.425217e+08,3
1,982093.34,6523781.04,1612.30,423,2,2,1,0,2,0,0,0,0,20.988001,7,5264,3.425217e+08,3
2,982091.22,6523780.56,1613.96,807,3,3,1,0,3,0,0,0,0,21.054001,7,5264,3.425217e+08,3
3,982089.88,6523780.25,1617.51,430,3,3,1,0,5,0,0,0,0,21.120001,7,5264,3.425217e+08,3
4,982086.91,6523779.58,1617.06,618,3,3,1,0,3,0,0,0,0,21.186001,7,5264,3.425217e+08,3


In [3]:
# Lambert 93 (EPSG:2154) → UTM Zone 32N (EPSG:32632)
transformer = Transformer.from_crs("epsg:2154", "epsg:32632", always_xy=True)
# Convert X, Y coordinates
utm_easting, utm_northing = transformer.transform(df_ign["X"].values, df_ign["Y"].values)
# Add transformed coordinates to DataFrame
df_ign["UTM_Easting"] = utm_easting
df_ign["UTM_Northing"] = utm_northing

In [4]:
print(f"number of liDAR data for one Tile: {len(df_ign)} points")

number of liDAR data for one Tile: 33210853 points


In [5]:
df_ign = pd.DataFrame(data={"x": df_ign["UTM_Easting"],
                            "y":df_ign["UTM_Northing"],
                            "z": df_ign["Z"],
                            'classification': df_ign["Classification"]
                            })

In [6]:
df_ign.head()

,x,y,z,classification
0,315679.346779,5.069587e+06,1617.46,2
1,315686.236590,5.069588e+06,1612.30,2
2,315684.084498,5.069587e+06,1613.96,3
3,315682.723714,5.069587e+06,1617.51,5
4,315679.708940,5.069587e+06,1617.06,3


In [7]:
# Get unique classification values
unique_classes = df_ign["classification"].unique()
# Generate unique colors (normalized to [0, 1] for Open3D)
num_classes = len(unique_classes)
cmap = plt.cm.get_cmap("tab10", num_classes)  # Choose a colormap with enough distinct colors
class_colors = {cls: tuple((np.array(cmap(i)[:3]) * 255).astype(int)) for i, cls in enumerate(unique_classes)}

/tmp/ipykernel_2319670/3779155000.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap("tab10", num_classes)  # Choose a colormap with enough distinct colors


In [8]:
# Map colors to classification values
df_ign["color"] = df_ign["classification"].map(class_colors)

In [9]:
sample_fraction = 5
lenght = int(5 * len(df_ign) / 100)
df_ign = df_ign[:lenght]
print(len(df_ign))

1660542


In [10]:
# Stack the UTM coordinates and DSM values into a single array
points = np.column_stack((df_ign['x'], df_ign['y'], df_ign['z'].values))

# Convert the RGB color tuples to float values in the range [0, 1]
colors = np.array(df_ign['color'].apply(lambda x: np.array(x))) / 255.0

# Create Open3D point cloud object
point_cloud = o3d.geometry.PointCloud()

# Set the points for the point cloud
point_cloud.points = o3d.utility.Vector3dVector(points)

# Set the colors for the point cloud
point_cloud.colors = o3d.utility.Vector3dVector(colors)
# Save the point cloud to a PLY file
o3d.io.write_point_cloud("ign_lidar_4_perc.ply", point_cloud)

True